<a href="https://colab.research.google.com/github/tiwari-arpit/nlp/blob/main/distilbert-finetuned-hindi-ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Libraries (Setup)

In [1]:
!pip install datasets
!pip install -U transformers
!pip install -U accelerate
!pip install seqeval
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 52.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.3
    Uninstalling transformers-4.48.3:
      Successfully uninstalled transformers-4.48.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 342.1/342.1 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd

# Dataset

In [3]:
# Dataset source: https://huggingface.co/datasets/ai4bharat/naamapadam

from datasets import load_dataset
lang = 'hi'
hindi_data = load_dataset('ai4bharat/naamapadam',lang)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.65k [00:00<?, ?B/s]

naamapadam.py:   0%|          | 0.00/2.86k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/93.3M [00:00<?, ?B/s]

0001.parquet:   0%|          | 0.00/5.48M [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/93.5k [00:00<?, ?B/s]

0000.parquet:   0%|          | 0.00/1.37M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/985787 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/867 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13460 [00:00<?, ? examples/s]

In [4]:
hindi_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 13460
    })
})

In [5]:
hindi_data['train'].to_pandas()

,tokens,ner_tags
0,"[सेक्टर, 55/56, के, एसएचओ, अरविंद, कुमार, ने, ...","[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[मध्य, रेलवे, एवं, पश्चिम, रेलवे, के, अधिकारिय...","[3, 4, 4, 4, 4, 0, 0, 0, 0, 0, 0, 3, 4, 0, 3, ..."
2,"[जाने, -, माने, वैज्ञानिक, सिवान, के, ., को, भ...","[0, 0, 0, 0, 1, 2, 0, 0, 3, 4, 4, 4, 0, 0, 0, ..."
3,"[यह, सूर्य, ग्रहण, भारत, में, भी, दिखेगा, .]","[0, 0, 0, 5, 0, 0, 0, 0]"
4,"[ज्ञापन, में, कहा, गया, है, कि, सीपीडब्ल्यूडी,...","[0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
985782,"[पाकिस्तानी, टीवी, होस्ट, नादिया, खान, ने, हॉल...","[0, 0, 0, 1, 2, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
985783,"[भारतीय, आयात, -, निर्यात, बैंक, ,, भारतीय, स्...","[0, 3, 0, 0, 0, 0, 0, 3, 0, 0, 3, 4, 4, 0, 3, ..."
985784,"[दिल्ली, की, पटियाला, हाउस, अदालत, ने, IRCTC, ...","[5, 0, 3, 4, 4, 0, 3, 0, 0, 0, 0, 0, 0, 0, 5, ..."
985785,"[आधारभूत, सुविधाओं, के, कार्यदल, ने, भारतीय, ह...","[0, 3, 4, 4, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [6]:
tags = hindi_data['train'].features['ner_tags'].feature

def create_tag_name(batch):
  tag_name = {'ner_tags_str': [ tags.int2str(idx) for idx in batch['ner_tags']]}
  return tag_name

In [7]:
hindi_data = hindi_data.map(create_tag_name)

Map:   0%|          | 0/985787 [00:00<?, ? examples/s]

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

Map:   0%|          | 0/13460 [00:00<?, ? examples/s]

In [8]:
hindi_data['train'].to_pandas().iloc[0]

,0
tokens,"[सेक्टर, 55/56, के, एसएचओ, अरविंद, कुमार, ने, ..."
ner_tags,"[0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
ner_tags_str,"[O, O, O, O, B-PER, I-PER, O, O, O, O, O, O, O..."


# Model

**Load Pre-trained Model for Tokenization**

In [9]:
from transformers import AutoTokenizer

model_checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/465 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

**Tokenize all texts and align the labels with them**

In [10]:
def align_labels_with_tokens(labels, word_ids):
  new_labels = []
  current_word=None
  for word_id in word_ids:
    if word_id != current_word:
      current_word = word_id
      label = -100 if word_id is None else labels[word_id]
      new_labels.append(label)

    elif word_id is None:
      new_labels.append(-100)

    else:
      label = labels[word_id]

      if label%2==1:
        label = label + 1
      new_labels.append(label)

  return new_labels

In [11]:
def tokenize_and_align_labels(examples):
  tokenized_inputs = tokenizer(examples['tokens'],truncation=True,is_split_into_words=True)
  all_labels = examples['ner_tags']
  new_labels = []
  for i, labels in enumerate(all_labels):
    word_ids = tokenized_inputs.word_ids(i)
    new_labels.append(align_labels_with_tokens(labels,word_ids))

  tokenized_inputs['labels'] = new_labels
  return tokenized_inputs

In [12]:
tokenized_data = hindi_data.map(tokenize_and_align_labels,batched=True,remove_columns=hindi_data['train'].column_names)

Map:   0%|          | 0/985787 [00:00<?, ? examples/s]

Map:   0%|          | 0/867 [00:00<?, ? examples/s]

Map:   0%|          | 0/13460 [00:00<?, ? examples/s]

In [13]:
tokenized_data

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 985787
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 867
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 13460
    })
})

**Create Data Collator and Metrics**

In [14]:
from transformers import DataCollatorForTokenClassification
data_colator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [15]:
batch = data_colator([tokenized_data['train'][i] for i in range(2)])
batch

{'input_ids': tensor([[  101,   100,  3731,   120,  4376,   607, 28535,   100,   100,   607,
         28534, 28522, 28531, 28524,   615, 28535,   617, 28515, 28531, 28523,
         28531,   607, 28532,   100,   619, 28531, 28522, 28525, 28535,   619,
         28535, 28508,   100,   607, 28533,   614, 28531, 28524, 28531,  3413,
          1545,   118,   100,   113,   100,   114,   607, 28535,   612, 28530,
         28515,   619, 28531, 28522, 28525, 28531,   613, 28524, 28537, 28513,
           607, 28524,   622, 28532, 28523, 28531,   608, 28523, 28531,   100,
           635,   102,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0],
        [  101,   619, 28517, 28537, 28523,   621, 28535, 28525, 28526, 28535,
           100,   616, 28527, 28537, 28512, 28532, 28522,   621, 28535, 28525,
         28526, 28535,   607, 28535,   100,   607, 28535,   100,   100,   118,
           100,   100,   117,   608, 

In [16]:
import evaluate
metric = evaluate.load('seqeval')

In [17]:
ner_feature = hindi_data['train'].features['ner_tags']
label_names = ner_feature.feature.names

In [18]:
id2label = {i:label for i, label in enumerate(label_names)}
label2id = {label:i for i, label in enumerate(label_names)}

In [19]:
print(id2label)

{0: 'O', 1: 'B-PER', 2: 'I-PER', 3: 'B-ORG', 4: 'I-ORG', 5: 'B-LOC', 6: 'I-LOC'}


In [20]:
import numpy as np

def compute_metrics(eval_preds):
  logits, labels = eval_preds

  predictions = np.argmax(logits, axis=-1)

  true_labels = [[label_names[l] for l in label if l!=-100] for label in labels]

  true_predictions = [[label_names[p] for p,l in zip(prediction, label) if l!=-100]
                      for prediction, label in zip(predictions, labels)]

  all_metrics = metric.compute(predictions=true_predictions, references=true_labels)

  return {"precision": all_metrics['overall_precision'],
          "recall": all_metrics['overall_recall'],
          "f1": all_metrics['overall_f1'],
          "accuracy": all_metrics['overall_accuracy']}

**Load Pre-trained Model for Token Classifiaction.**

In [21]:
from transformers import AutoModelForTokenClassification
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model.safetensors:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [22]:
model.config.num_labels

7

**Set Training Arguments**

In [23]:
# Training on 50% data for 1 epoch
from sklearn.model_selection import train_test_split
train_size = 0.5
train_dataset = tokenized_data["train"].train_test_split(train_size=train_size)

train_dataset = train_dataset["train"]

In [26]:
from transformers import TrainingArguments

args = TrainingArguments(output_dir = "./distilbert-finetuned-hindi-ner",
                         evaluation_strategy = "epoch",
                         save_strategy="epoch",
                         learning_rate = 2e-5,
                         num_train_epochs=1,
                         weight_decay=0.01)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


**Training**

In [27]:
from transformers import Trainer

trainer = Trainer(model = model,
                  args = args,
                  train_dataset = train_dataset,
                  eval_dataset = tokenized_data['test'],
                  data_collator = data_colator,
                  compute_metrics = compute_metrics,
                  tokenizer = tokenizer)
trainer.train()

<ipython-input-27-c51e5438a738>:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model = model,


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.263400,0.204694,0.571720,0.580612,0.576132,0.927248


TrainOutput(global_step=61612, training_loss=0.3012761005386009, metrics={'train_runtime': 6538.5114, 'train_samples_per_second': 75.383, 'train_steps_per_second': 9.423, 'total_flos': 1.5963066104685282e+16, 'train_loss': 0.3012761005386009, 'epoch': 1.0})

In [31]:
trainer.save_model("./distilbert-finetuned-hindi-ner")

In [29]:
from transformers import pipeline

token_classifier = pipeline(
    "token-classification",
    model="./distilbert-finetuned-ner",
    aggregation_strategy="simple"
)

token_classifier("इटली की पीएम जॉर्जिया मेलोनी ने पूरी दुनिया के लेफ्टिस्ट लीडर्स को पाखंडी बताया है। उन्होंने कहा कि दुनियाभर में मोदी, ट्रम्प और मेरे जैसे दक्षिणपंथी नेताओं के उभरने से सारे लेफ्टिस्ट नेता परेशान हो गए हैं|")

Device set to use cuda:0


[{'entity_group': 'PER',
  'score': 0.84545565,
  'word': 'जॉर्जिया मेलोनी',
  'start': 13,
  'end': 28},
 {'entity_group': 'PER',
  'score': 0.93689823,
  'word': 'मोदी',
  'start': 113,
  'end': 117}]